<h1 align="center"> Homework 16.3 </h1>
<h1 align="center">Caelan Osman </h1>
<h1 align="center">March 8, 2022 </h1>

In [15]:
import numpy as np
from tensorflow.keras.datasets.fashion_mnist import load_data
from sklearn.ensemble import GradientBoostingClassifier
import time

## Exercise 16.10

In [2]:
def JL_Bound(N, eps):
    """
    Computes the Johnson-Lindenstrauss bound k

    parameters:
        n (int): number of points
        eps (float): maximal allowed distortion of the distance
    
    Returns:
        k (int): the dimensional bound
    """

    return int(np.ceil(24*np.log(N)/(3*eps**2 - 2*eps**3)))

def preserves_distance(X, A, eps, ord=2):
    """
    This checks if a linear transformation preserves distance between points
    up to some distortion epsilon. 

    Parameters:
        X: np.ndarray ((N, d)) N is the number of points d is the dimension
        A: np.ndarray ((k, d)) A: R^d - R^k
        eps: (float) distortion constatnd
        ord: (int) order of norm

    Returns:
        boolean: if the linear transformation preserves distance
    """

    preserves = True
    N, _ = X.shape
    for i in range(N):
        for j in range(i+1, N):

            # check preservation
            lower_bound = (1 - eps)*np.linalg.norm(X[i] - X[j], ord=ord)**2 <= np.linalg.norm(A@X[i] -A@X[j], ord=ord)**2
            upper_bound = (1 + eps)*np.linalg.norm(X[i] - X[j], ord=ord)**2 >= np.linalg.norm(A@X[i] -A@X[j], ord=ord)**2
            preserves = lower_bound and upper_bound

            if not preserves:
                return False

    return True

def random_points(N, d=10**5):
    """
    Uniformly draws N random points from R^d
    """
    return np.random.uniform(low=-10000, high=10000, size=(N, d))

def random_projection(k, d=10**5):
    """ 
    Constructs a random projection where each entries is drawn from
    N(0, 1/k)
    """
    return np.random.normal(loc=0, scale=np.sqrt(1/k), size=(k, d))

def run_projection_test():
    """
    Tests if a random projection preserves distance for a random points
    """

    N_vals = [3, 10, 100]
    Eps = [0.7, 0.5, 0.1]

    Failures_before_pass = {(N, eps) : -1 for N in N_vals for eps in Eps}

    for N in N_vals:
        X =  random_points(N)
        for eps in Eps:
            passed = False
            k = JL_Bound(N, eps)
            while not passed:
                A = random_projection(k)
                passed = preserves_distance(X, A, eps)

                Failures_before_pass[(N, eps)] += 1

    return Failures_before_pass

FBP = run_projection_test()

for key in list(FBP.keys()):
    print("N: ", key[0], ",eps: ", key[1])
    print('Number of failures before passing: ', FBP[key])
    print()

N:  3 ,eps:  0.7
Number of failures before passing:  0

N:  3 ,eps:  0.5
Number of failures before passing:  0

N:  3 ,eps:  0.1
Number of failures before passing:  0

N:  10 ,eps:  0.7
Number of failures before passing:  0

N:  10 ,eps:  0.5
Number of failures before passing:  0

N:  10 ,eps:  0.1
Number of failures before passing:  1

N:  100 ,eps:  0.7
Number of failures before passing:  0

N:  100 ,eps:  0.5
Number of failures before passing:  0

N:  100 ,eps:  0.1
Number of failures before passing:  0



As we can see only one combination $N = 10$ and $\varepsilon = 0.1$ failed before passing. Every other combination passed first try. 


## Exercise 16.14

### Part 1

In [39]:
def random_projection_mnist():

    # load data
    (x_train, y_train), (x_test, y_test) = load_data()
    # convert to 728 flattened images
    input_dim = 784 #28*28
    X_train = x_train.reshape(60000, input_dim)
    X_test = x_test.reshape(10000, input_dim)

    # scale images
    X_train = X_train/255
    X_test = X_test/255

    # randomly pick which features to keep
    keep_features = np.random.choice(np.arange(0, input_dim), size=int(input_dim/10), replace=False)
    X_train = X_train[:, keep_features]
    X_test = X_test[:, keep_features]

    # we now change the sign on half of them
    change_sign = np.random.choice(np.arange(0, X_train.shape[1]), size = int(X_train.shape[1]/2), replace=False)
    X_train[:, change_sign] *= -1
    X_test[:, change_sign] *= -1

    return X_train, X_test

def unmodified_data():
    # load data
    (x_train, y_train), (x_test, y_test) = load_data()
    # convert to 728 flattened images
    input_dim = 784 #28*28
    X_train = x_train.reshape(60000, input_dim)
    X_test = x_test.reshape(10000, input_dim)

    # scale images
    X_train = X_train/255
    X_test = X_test/255

    return X_train, y_train, X_test, y_test


### Part 2

In [40]:
def train_gradient_boosted():
    # get data
    X_train, y_train, _, _ = unmodified_data()
    X_train_projection, _ = random_projection_mnist()
    N = X_train.shape[0]
    training_set = np.random.choice(np.arange(0, N), size = int(N/10),  replace=False) 

    # time GBT on original data
    start = time.time()
    clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,  max_depth=1, random_state=0).fit(X_train[training_set], y_train[training_set])
    end = time.time()
    unmodified_time = end - start

    # time GBT on modified data
    start = time.time()
    clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,  max_depth=1, random_state=0).fit(X_train_projection[training_set], y_train[training_set])
    end = time.time()
    modified_time = end - start

    return unmodified_time, modified_time

unmodified_time, modified_time = train_gradient_boosted()

print('Time to train GBT on original data: ', unmodified_time)
print('Time to train GBT on randomly projected data: ', modified_time)

Time to train GBT on original data:  338.0774824619293
Time to train GBT on randomly projected data:  34.56087255477905


### Part 3

In [41]:
def score_gradient_boosted():
    # get and modify data

    X_train, y_train, X_test, y_test = unmodified_data()
    X_train_projection, X_test_projection = random_projection_mnist()
    N = X_train.shape[0]
    training_set = np.random.choice(np.arange(0, N), size = int(N/10), replace=False) 

    # train and score GBT on original data
    clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,  max_depth=1, random_state=0).fit(X_train[training_set], y_train[training_set])
    unmodified_score = clf.score(X_test, y_test)

    # train and score GBT on modified data
    clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,  max_depth=1, random_state=0).fit(X_train_projection[training_set], y_train[training_set])
    modified_score = clf.score(X_test_projection, y_test)

    return unmodified_score, modified_score

unmodified_score, modified_score = score_gradient_boosted()

print('GBT score on original data: ', unmodified_score)
print('GBT score on randomly projected data: ', modified_score)


GBT score on original data:  0.6616
GBT score on randomly projected data:  0.7612


### Part 4

In [34]:
def time_PCA():
    # get and modify data
    X_train, y_train, X_test, y_test = unmodified_data()

    N = X_train.shape[0]
    training_set = np.random.choice(np.arange(0, N), size = int(N/10),  replace=False) 
    X_train = X_train[training_set]

    # compute and time PCA on original data
    start = time.time()
    _, _, VH = np.linalg.svd(X_train)
    X_train @ VH[:20, :].T
    _, _, VH = np.linalg.svd(X_test)
    X_test @ VH[:20, :].T
    end = time.time()
    PCA_time = end - start


    return PCA_time


print('Time to compute SVD on original data: ', time_PCA())

Time to compute SVD on original data:  8.219223737716675


### Part 5 and 6

In [36]:
def PCA_GBT_accuracy_time():

    # set up training data
    X_train, y_train, X_test, y_test = unmodified_data()
    X_train_projection, X_test_projection = random_projection_mnist()

    N = X_train.shape[0]
    training_set = np.random.choice(np.arange(0, N), size = int(N/10),  replace=False) 

    X_train = X_train[training_set]
    X_train_projection = X_train_projection[training_set]

    start = time.time()
    # compute PCA
    _, _, VH = np.linalg.svd(X_train)
    PCA_train = X_train @ VH[:20, :].T
    _, _, VH = np.linalg.svd(X_test)
    PCA_test = X_test @ VH[:20, :].T
    # train tree
    clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,  max_depth=1, random_state=0).fit(PCA_train, y_train[training_set])
    end = time.time()
    # time and score unmodified data
    unmodified_time = end - start
    _, _, VH = np.linalg.svd(X_test)
    unmodified_score = clf.score(PCA_test, y_test)

    start = time.time()
    # train tree
    clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,  max_depth=1, random_state=0).fit(X_train_projection, y_train[training_set])
    end = time.time()
    # time and score modified data
    modified_time = end - start
    modified_score = clf.score(X_test_projection, y_test)

    return unmodified_time, unmodified_score, modified_time, modified_score

unmodified_time, unmodified_score, modified_time, modified_score = PCA_GBT_accuracy_time()

print('Time to compute SVD and train GBT on original data: ', unmodified_time)
print('Score: ', unmodified_score)

print('Time to train GBT on projected data: ', modified_time)
print('Score: ', modified_score)

Time to compute SVD and train GBT on original data:  24.586934804916382
Score:  0.1234
Time to train GBT on projected data:  18.622740745544434
Score:  0.6541
